In [ ]:
import sys

sys.path.append('../')

In [ ]:
from script.saving import save_model
from script.config import FINAL_PATH, MODEL_PATH
from modeling.svc import SupportVectorClassifier

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(FINAL_PATH)

In [ ]:
X = df.drop("User Behavior Class", axis=1)
y = df["User Behavior Class"]

In [ ]:
model = SupportVectorClassifier()

In [ ]:
model.train(X, y)

In [ ]:
y_pred = model.prediction()

In [ ]:
metrics = model.evaluate()

In [ ]:
metrics.display()

In [ ]:
save_model(model, MODEL_PATH)

In [ ]:
import sys

sys.path.append("../")

In [ ]:
from script.feature import new_features
from script.config import MODEL_PATH
from pydantic import BaseModel
from fastapi import FastAPI
import pandas as pd
import numpy as np
import joblib

In [ ]:
class InputData(BaseModel):
    f1: int
    f2: int
    f3: float
    f4: float
    f5: float
    f6: int
    f7: float

In [ ]:
FEATURE_NAMES = [
        "Device Model",
        "Operating System",
        "App Usage Time (min/day)",
        "Screen On Time (hours/day)",
        "Battery Drain (mAh/day)",
        "Number of Apps Installed",
        "Data Usage (MB/day)",
]

In [ ]:
model = joblib.load(MODEL_PATH)

In [ ]:
data = InputData(f1=1, f2=2, f3=3, f4=4, f5=5, f6=6, f7=7)

In [ ]:
data_dict = {
    "Device Model": data.f1,
    "Operating System": data.f2,
    "App Usage Time (min/day)": data.f3,
    "Screen On Time (hours/day)": data.f4,
    "Battery Drain (mAh/day)": data.f5,
    "Number of Apps Installed": data.f6,
    "Data Usage (MB/day)": data.f7,
}

In [ ]:
input_data = pd.DataFrame(data_dict, index=[0])

In [ ]:
new_feature = pd.DataFrame(new_features(input_data))

In [ ]:
features = pd.concat([input_data, new_feature], axis=1)

In [ ]:
features

In [ ]:
model.predict_client(features).tolist()[0]

In [ ]:
class ModelPredictor:
    FEATURE_MAPPING = {
        "Device Model": "f1",
        "Operating System": "f2",
        "App Usage Time (min/day)": "f3",
        "Screen On Time (hours/day)": "f4",
        "Battery Drain (mAh/day)": "f5",
        "Number of Apps Installed": "f6",
        "Data Usage (MB/day)": "f7"
    }

    def __init__(self, model_path: str) -> None:
        self.model = joblib.load(model_path)

    def _create_input_dataframe(self, data: InputData) -> pd.DataFrame:
        return pd.DataFrame({
            col_name: getattr(data, attr_name)
            for col_name, attr_name in self.FEATURE_MAPPING.items()
        }, index=[0])

    def predict(self, data):
        base_features = self._create_input_dataframe(data)
        derived_features = pd.DataFrame(new_features(base_features))
        complete_features = pd.concat([base_features, derived_features], axis=1)
        return self.model.predict_client(complete_features).tolist()[0]

In [ ]:
predictor = ModelPredictor(MODEL_PATH)

In [ ]:
predictor.predict(data)